In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Final Year Project')
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1DUsC04vbUs9uo8LysHyaB0N8klv6t5Ln/Final Year Project


In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
import numpy as np
import csv
import json

In [ ]:
cd '/content/drive/MyDrive/Final Year Project'

/content/drive/.shortcut-targets-by-id/1DUsC04vbUs9uo8LysHyaB0N8klv6t5Ln/Final Year Project


In [ ]:
#fp = open('Slake1.0/train_data.csv', 'r')
fp = open('Slake1.0/train.json', 'r')
reader = json.load(fp)

In [ ]:
modality_data = []
for item in reader:
  if item['answer'] == '':
    continue
  else:
    if item['content_type'] == 'Modality' and item['q_lang'] == 'en':
      modality_data.append(item)

In [ ]:
len(modality_data)

534

In [ ]:
modality_data[0]

{'answer': 'MRI',
 'answer_type': 'OPEN',
 'base_type': 'vqa',
 'content_type': 'Modality',
 'img_id': 1,
 'img_name': 'xmlab1/source.jpg',
 'location': 'Abdomen',
 'modality': 'MRI',
 'q_lang': 'en',
 'qid': 0,
 'question': 'What modality is used to take this image?',
 'triple': ['vhead', '_', '_']}

In [ ]:
fp.close()

In [ ]:
modality_images = []
for item in modality_data:
  modality_images.append(item['img_name'])

In [ ]:
len(modality_images)

534

In [ ]:
modality_images[0]

'xmlab1/source.jpg'

In [ ]:
images = modality_images

The next few blocks of code is for converting the images into vgg vectors and storing them in files. Already been done no need to re-execute

np.array of index i stores vgg vectorized form of image xmlab{i}/source.jpg

In [ ]:
vgg_model = VGG16(include_top=False)
#f = open('Image Classifier/vgg_train_data.csv', 'w')
img_path = 'Slake1.0/imgs/'

In [ ]:
images = []

for i in range(642):
    images.append(f"xmlab{i}/source.jpg")

Below two blocks of code used for storing the vgg outputs of all the images into a file. It has been done previously so no need to run again.

In [ ]:
#image_vectors = []
'''
for i in range(len(images)):
    full_img_path = img_path + images[i]
    img = image.load_img(full_img_path, color_mode='rgb', target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    vgg_out = vgg_model(img).numpy()
    vgg_out = vgg_out.flatten()

    image_vectors.append(vgg_out)'''

In [ ]:
vgg_image_vecs = np.array(image_vectors)

In [ ]:
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1DUsC04vbUs9uo8LysHyaB0N8klv6t5Ln/Final Year Project


In [ ]:
cd '/content/drive/MyDrive/Final Year Project/Image Classifier'

/content/drive/.shortcut-targets-by-id/1DUsC04vbUs9uo8LysHyaB0N8klv6t5Ln/Final Year Project/Image Classifier


In [ ]:
#np.savetxt('vgg_train_data_modality.csv', vgg_image_vecs)

Loads the saved vgg weights of images into a numpy array.

In [ ]:
new_vgg = np.loadtxt('vgg_train_data.csv')

In [ ]:
cd '/content/drive/MyDrive/Final Year Project'

/content/drive/.shortcut-targets-by-id/1DUsC04vbUs9uo8LysHyaB0N8klv6t5Ln/Final Year Project


new_vgg is a 642x25088 numpy matrix consisting of the vectorized images in the SLAKE dataset. Each one of these images must be transferred to the reduction model to obtain a 642x512 matrix, with row i containing the vectorized form of image i

Principle Component Analysis has been performed to reduce the number of features from 25088 to 512. Accuracy of model yet to be tested

In [ ]:
'''from sklearn.decomposition import PCA

pca = PCA(512)
pca.fit(new_vgg)
final_vgg = pca.transform(new_vgg)'''

final_vgg is the image feature matrix from which the corresponding image feature will be selected and given for feature fusion and classification

In [ ]:
#np.savetxt('Image Classifier/final_vgg_vectors.csv', final_vgg)

In [ ]:
final_vgg = new_vgg

In [ ]:
# def get_reduction_model():
#     red_1 = Input(shape=(7,3584))
#     red_1_5 = Flatten()(red_1)
#     red_2 = Dense(units = 4096, activation = 'relu')(red_1_5)
#     red_3 = Dense(units = 2048, activation = 'relu')(red_2)
#     red_4 = Dense(units = 512, activation = 'relu')(red_3)

#     return Model(inputs=red_1, outputs=red_4)

# red_model = get_reduction_model()

# Question Classifier

In [ ]:
import json

1. CT Questions
2. MRI Questions
3. X-Ray Questions

In [ ]:
train_questions = []
train_answers = []

test_questions = []
test_answers = []

valid_questions = []
valid_answers = []

with open('Slake1.0/train.json') as f:
    train_data = json.load(f)

    for item in train_data:
        #print(item)
        if item['answer'] == '':
            continue
        else:
            if item['content_type'] == 'Modality' and item['q_lang']=='en':
                train_questions.append(item['question'])
                train_answers.append(item['answer'])

with open('Slake1.0/test.json') as f:
    test_data = json.load(f)

    for item in test_data:
        if item['answer'] == '':
            continue
        else:
            if item['content_type'] == 'Modality' and item['q_lang']=='en':
                test_questions.append(item['question'])
                test_answers.append(item['answer'])

with open('Slake1.0/validate.json') as f:
    valid_data = json.load(f)

    for item in valid_data:
        if item['answer'] == '':
            continue
        else:
            if item['content_type'] == 'Modality' and item['q_lang']=='en':
                valid_questions.append(item['question'])
                valid_answers.append(item['answer'])

In [ ]:
len(train_answers)

534

In [ ]:
len(train_answers)
train_answers_index = list(set(train_answers))
print(train_answers_index)

train_answer_dict = {}

for index, answer in enumerate(train_answers_index):
    train_answer_dict[answer] = index

['T1', 'T2', 'Yes', 'X-Ray', 'MRI', 'No', 'CT']


In [ ]:
print(len(train_answers_index))

7


In [ ]:
from tensorflow.keras.utils import to_categorical

y_cat = []

for answer in train_answers:
    y_cat.append(to_categorical(train_answer_dict[answer], num_classes=139))

In [ ]:
y_cat = np.array(y_cat)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [ ]:
t = Tokenizer()
t.fit_on_texts(train_questions)
vocab_size = len(t.word_index) + 1
print(t.word_index)

maxlen=16

{'this': 1, 'is': 2, 'image': 3, 'what': 4, 'modality': 5, 'used': 6, 'to': 7, 'take': 8, 'in': 9, 'weighting': 10, 'mri': 11, 'the': 12, 'mr': 13, 'taken': 14, 'shown': 15, 'of': 16, 'ct': 17, 'was': 18, 'an': 19, 'a': 20, 'type': 21, 'weighted': 22, 'x': 23, 'ray': 24, 'via': 25, 'with': 26, 'or': 27, 't2': 28, 't1': 29, 'how': 30, 'which': 31, 'scan': 32, 'imaging': 33, 'about': 34, 'medical': 35, 'kind': 36, 'generated': 37, 'liver': 38, 'normal': 39}


In [ ]:
def obtain_padded_questions(questions, maxlen):
    encoded = t.texts_to_sequences(questions)

    padded = pad_sequences(encoded, padding='post', maxlen=maxlen)
    padded = np.array(padded)

    return padded

In [ ]:
embeddings_index = dict()

f = open('Question Classifier/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if word.endswith("'s"):
    embedding_vector = embeddings_index.get(word[:-2])
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
  else:
    print(word)

In [ ]:
##########################################################

In [ ]:
from keras.layers import LSTM, Dropout
import tensorflow.keras.utils as ku

In [ ]:
model_ques = Sequential([
    Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=16, trainable=False),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    Flatten()
])

In [ ]:
model_ques.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           4000      
                                                                 
 lstm (LSTM)                 (None, 16, 50)            30200     
                                                                 
 dropout (Dropout)           (None, 16, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 16, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 16, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 16, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 16, 50)            0

In [ ]:
# class_inp = Input(shape=(16,))
# class_mid = model_ques(class_inp)
# class_out = Dense(units=128, activation='softmax')(class_mid)

# class_ques_model = Model(inputs=class_inp, outputs=class_out)

In [ ]:
# class_ques_model.summary()

In [ ]:
# class_ques_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#class_ques_model.fit(padded, y_cat, batch_size=32, epochs=10)

In [ ]:
def obtain_question_vectors(questions:list, max:int)->np.ndarray:
    sequence = t.texts_to_sequences(questions)
    padded = pad_sequences(sequence, maxlen=max, padding='post')
    output = model_ques(padded)

    return output.numpy()

In [ ]:
sample = ["What modality is this picture?"]
sequence = t.texts_to_sequences(sample)
pad_sequence = pad_sequences(sequence, maxlen=maxlen, padding='post')

In [ ]:
pad_sequence

array([[4, 5, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
question_vectors = obtain_question_vectors(train_questions, maxlen)

In [ ]:
question_vectors.shape

(534, 800)

In [ ]:
pca_ques = PCA(512)
pca_ques.fit(question_vectors)
final_ques_vec = pca_ques.transform(question_vectors)
final_ques_vec.shape

(4212, 512)

final_ques_vec is a n_questionsx512 numpy array representing all the questions in the training dataset in their vectorized form

In [ ]:
def obtain_final_question_vectors(questions):
    ques_vecs = obtain_question_vectors(questions, 16)
    
    # pca_ques = PCA(512)
    # pca_ques.fit(question_vectors)
    # final_ques_vec = pca_ques.transform(question_vectors)
    # final_ques_vec.shape

    return ques_vecs

In [ ]:
np.savetxt('Question Classifier/final_question_vectors.csv', final_ques_vec)

In [ ]:
final_ques_vec = np.loadtxt('Question Classifier/final_question_vectors.csv')

In [ ]:
final_ques_vec.shape

(4212, 512)

In [ ]:
def concatenation(data_filename, final_ques_vec, final_vgg):
    img_ques_concat = []
    i=0

    with open(f'Slake1.0/{data_filename}.json') as f:
        train_data = json.load(f)

        for item in train_data:
            if item['answer'] == '':
                continue
            else:
                if item['content_type'] == 'Modality' and item['q_lang']=='en':
                    image_id = item['img_id']#item['image'][5:][:-11]
                    image_vec = final_vgg[int(image_id)]
                    question_vec = final_ques_vec[i]

                    img_ques_concat.append(np.concatenate((image_vec, question_vec)))

                    i+=1
    
    return np.array(img_ques_concat)

img_ques_concat is the concatenated joint representation for all image and question pairs. It is a 4212x1024 array. Can be replaced with element-wise product or attention networks.

In [ ]:
def train_model(img_ques_concat, y_cat, valid_x, valid_y, num):

    classify_model = Sequential([
        Input(shape=(25888,)),
        Dense(units=1024, activation='relu'),
        Dense(units=512, activation='relu'),
        Dense(units=256, activation='relu'),
        Dense(units=num, activation='softmax')
    ])

    classify_model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = classify_model.fit(img_ques_concat, y_cat, batch_size=32, epochs=100, validation_data=(valid_x, valid_y))

    return classify_model, history

In [ ]:
def obtain_y_cat(answers, answer_dict, num):

    y_cat = []

    for answer in answers:
        y_cat.append(to_categorical(answer_dict[answer], num_classes=num))

    return np.array(y_cat)

In [ ]:
train_answers_index = list(set(train_answers + test_answers + valid_answers))
num = len(train_answers_index)

train_answer_dict = {}

for index, answer in enumerate(train_answers_index):
    train_answer_dict[answer] = index

#train_data
final_ques_vec_train = obtain_final_question_vectors(train_questions)
img_ques_concat_train = concatenation('train', final_ques_vec_train, final_vgg)
y_cat_train = obtain_y_cat(train_answers, train_answer_dict, num)

#valid_data
final_ques_vec_valid = obtain_final_question_vectors(valid_questions)
img_ques_concat_valid = concatenation('validate', final_ques_vec_valid, final_vgg)
y_cat_valid = obtain_y_cat(valid_answers, train_answer_dict, num)

#test_data
final_ques_vec_test = obtain_final_question_vectors(test_questions)
img_ques_concat_test = concatenation('test', final_ques_vec_test, final_vgg)
y_cat_test = obtain_y_cat(test_answers, train_answer_dict, num)

In [ ]:
img_ques_concat_train.shape

(534, 25888)

In [ ]:
model, history = train_model(img_ques_concat_train, y_cat_train, img_ques_concat_valid, y_cat_valid, num)

Epoch 1/100
17/17 [==============================] - 4s 192ms/step - loss: 29.2043 - accuracy: 0.4064 - val_loss: 14.1315 - val_accuracy: 0.5000
Epoch 2/100
17/17 [==============================] - 3s 176ms/step - loss: 6.6136 - accuracy: 0.5730 - val_loss: 9.6922 - val_accuracy: 0.5273
Epoch 3/100
17/17 [==============================] - 3s 177ms/step - loss: 3.8072 - accuracy: 0.6124 - val_loss: 9.1176 - val_accuracy: 0.5455
Epoch 4/100
17/17 [==============================] - 3s 178ms/step - loss: 2.0707 - accuracy: 0.6891 - val_loss: 9.5293 - val_accuracy: 0.4818
Epoch 5/100
17/17 [==============================] - 3s 177ms/step - loss: 1.7063 - accuracy: 0.6723 - val_loss: 6.6941 - val_accuracy: 0.5727
Epoch 6/100
17/17 [==============================] - 3s 176ms/step - loss: 1.1155 - accuracy: 0.7154 - val_loss: 8.2854 - val_accuracy: 0.5273
Epoch 7/100
17/17 [==============================] - 3s 175ms/step - loss: 1.1864 - accuracy: 0.6910 - val_loss: 8.3943 - val_accuracy: 0.50

In [ ]:
train_accuracy = sum(history.history['accuracy'])/len(history.history['accuracy'])
valid_accuracy = sum(history.history['val_accuracy'])/len(history.history['val_accuracy'])

train_loss = sum(history.history['loss'])/len(history.history['loss'])
valid_loss = sum(history.history['val_loss'])/len(history.history['val_loss'])

In [ ]:
print(f"""
Train Accuracy: {train_accuracy*100}%
Valid Accuracy: {valid_accuracy*100}%
Train Loss: {train_loss}
Valid Loss: {valid_loss}
""")


Train Accuracy: 72.42322137951851%
Valid Accuracy: 54.12727329134941%
Train Loss: 0.8662857103347779
Valid Loss: 8.1982976770401



In [ ]:
model.evaluate(img_ques_concat_test, y_cat_test, batch_size=32)

4/4 [==============================] - 0s 41ms/step - loss: 5.3988 - accuracy: 0.5648


[5.39882755279541, 0.5648148059844971]